In [30]:
# coding: utf-8

import os
import sys
import json
import math
import numpy as np

Target = 'abz5'

file = open('./instances.json', "r" )
data = json.load(file)

instance = [ inst for inst in data if inst['name'] == Target ]
if(len(instance) == 0):
    raise Exception("There is no instance named %s" % Target)

instance = instance[0]
path = os.path.abspath("../%s" % instance['path'])
optimum = instance['optimum']

if( optimum is None ):
    if( instance['bounds'] is None ):
        optimum = "nan"
    else:
        optimum = instance['bounds']['lower']

sys.stdout.write('--instance %s --optimum %s' % (path,optimum))

--instance /home/fatima/Downloads/toDrive/AI 2024-2025/instances/abz5 --optimum 1234

84

In [31]:
instance

{'name': 'abz5',
 'jobs': 10,
 'machines': 10,
 'optimum': 1234,
 'path': 'instances/abz5'}

In [32]:
job_times = [[2, 1, 3], [4, 2, 1], [3, 3, 2]] # [0, 1, 2, 3, 4, 5, 6 ,7, 8] (how represented in the solution, flatten and re-enumerated)
# job 0: [2, 1, 3]
# Machine 1: 2, index 0 in the solution
# Machine 2: 1  index 1 in the solution
# Machine 3: 3  index 2 in the solution
# etc. same for the others

In [33]:
path_file = './instances/abz5'

# read raw data
with open(path_file) as f_o:
    data = f_o.read()
    lines = data.splitlines()
    print(lines)

jobs = np.zeros((10, 10))
lines = lines[5:]
lines, len(lines)

['#+++++++++++++++++++++++++++++', '# instance abz5', '#+++++++++++++++++++++++++++++', '# Adams, Balas, and Zawack 10x10 instance (Table 1, instance 5)', '10 10', '4 88 8 68 6 94 5 99 1 67 2 89 9 77 7 99 0 86 3 92', '5 72 3 50 6 69 4 75 2 94 8 66 0 92 1 82 7 94 9 63', '9 83 8 61 0 83 1 65 6 64 5 85 7 78 4 85 2 55 3 77', '7 94 2 68 1 61 4 99 3 54 6 75 5 66 0 76 9 63 8 67', '3 69 4 88 9 82 8 95 0 99 2 67 6 95 5 68 7 67 1 86', '1 99 4 81 5 64 6 66 8 80 2 80 7 69 9 62 3 79 0 88', '7 50 1 86 4 97 3 96 0 95 8 97 2 66 5 99 6 52 9 71', '4 98 6 73 3 82 2 51 1 71 5 94 7 85 0 62 8 95 9 79', '0 94 6 71 3 81 7 85 1 66 2 90 4 76 5 58 8 93 9 97', '3 50 0 59 1 82 8 67 7 56 9 96 6 58 4 81 5 59 2 96']


(['4 88 8 68 6 94 5 99 1 67 2 89 9 77 7 99 0 86 3 92',
  '5 72 3 50 6 69 4 75 2 94 8 66 0 92 1 82 7 94 9 63',
  '9 83 8 61 0 83 1 65 6 64 5 85 7 78 4 85 2 55 3 77',
  '7 94 2 68 1 61 4 99 3 54 6 75 5 66 0 76 9 63 8 67',
  '3 69 4 88 9 82 8 95 0 99 2 67 6 95 5 68 7 67 1 86',
  '1 99 4 81 5 64 6 66 8 80 2 80 7 69 9 62 3 79 0 88',
  '7 50 1 86 4 97 3 96 0 95 8 97 2 66 5 99 6 52 9 71',
  '4 98 6 73 3 82 2 51 1 71 5 94 7 85 0 62 8 95 9 79',
  '0 94 6 71 3 81 7 85 1 66 2 90 4 76 5 58 8 93 9 97',
  '3 50 0 59 1 82 8 67 7 56 9 96 6 58 4 81 5 59 2 96'],
 10)

In [39]:
for i in range(len(lines)-1):
    line = lines[i].split(' ')
    line = [int(num) for num in line]
    for j in range(0, len(line) -1 , 2):
        jobs[i][line[j]] = int(line[j+1])
    
jobs

array([[86., 67., 89., 92., 88., 99., 94., 99., 68., 77.],
       [92., 82., 94., 50., 75., 72., 69., 94., 66., 63.],
       [83., 65., 55., 77., 85., 85., 64., 78., 61., 83.],
       [76., 61., 68., 54., 99., 66., 75., 94., 67., 63.],
       [99., 86., 67., 69., 88., 68., 95., 67., 95., 82.],
       [88., 99., 80., 79., 81., 64., 66., 69., 80., 62.],
       [95., 86., 66., 96., 97., 99., 52., 50., 97., 71.],
       [62., 71., 51., 82., 98., 94., 73., 85., 95., 79.],
       [94., 66., 90., 81., 76., 58., 71., 85., 93., 97.],
       [50., 59., 82., 67., 56., 96., 58., 81., 59., 96.]])

In [37]:
for i in range(len(lines)):
    line = lines[i].split(' ')
    line = [int(num) for num in line]
    jobs[i] = line[1::2]

In [38]:
jobs

array([[88., 68., 94., 99., 67., 89., 77., 99., 86., 92.],
       [72., 50., 69., 75., 94., 66., 92., 82., 94., 63.],
       [83., 61., 83., 65., 64., 85., 78., 85., 55., 77.],
       [94., 68., 61., 99., 54., 75., 66., 76., 63., 67.],
       [69., 88., 82., 95., 99., 67., 95., 68., 67., 86.],
       [99., 81., 64., 66., 80., 80., 69., 62., 79., 88.],
       [50., 86., 97., 96., 95., 97., 66., 99., 52., 71.],
       [98., 73., 82., 51., 71., 94., 85., 62., 95., 79.],
       [94., 71., 81., 85., 66., 90., 76., 58., 93., 97.],
       [50., 59., 82., 67., 56., 96., 58., 81., 59., 96.]])

In [ ]:
optimum

In [14]:
from job_shop_lib.benchmarking import (
    load_all_benchmark_instances,
    load_benchmark_instance,
)

In [ ]:
benchmark_instances = load_all_benchmark_instances()
benchmark_instances

In [18]:
abz5 = load_benchmark_instance("abz5")

In [ ]:
abz5.metadata

In [ ]:
abz5.jobs